# Basic Model Training

We have the library CNN built and used in each model training. We simply use the model's application below.

In [1]:
from __future__ import print_function # Use a function definition from future version (say 3.x from 2.7 interpreter)
from IPython.display import Image
import matplotlib.pyplot as plt
import math
import numpy as np
import os
import PIL
import argparse
import sys
from CNN import create_reader,evaluate_model,train_and_evaluate,save_model,reader_train,reader_test,myimg


try:
    from urllib.request import urlopen
except ImportError:
    from urllib import urlopen

import cntk as C

if 'TEST_DEVICE' in os.environ:
    if os.environ['TEST_DEVICE'] == 'cpu':
        C.device.try_set_default_device(C.device.cpu())
    else:
        C.device.try_set_default_device(C.device.gpu(0))
        
model_name = "test_simple_model.dnn"
data_path = os.path.join('data', 'CIFAR-10')
import cntk.io.transforms as xforms
		

Reading map file: data\CIFAR-10\train_map.txt
Reading mean file: data\CIFAR-10\CIFAR-10_mean.xml
Reading map file: data\CIFAR-10\test_map.txt
Reading mean file: data\CIFAR-10\CIFAR-10_mean.xml


In [2]:
def create_basic_model(input, out_dims):
    with C.layers.default_options(init=C.glorot_uniform(), activation=C.relu):
        net = C.layers.Convolution((5,5), 32, pad=True)(input)
        net = C.layers.MaxPooling((3,3), strides=(2,2))(net)

        net = C.layers.Convolution((5,5), 32, pad=True)(net)
        net = C.layers.MaxPooling((3,3), strides=(2,2))(net)

        net = C.layers.Convolution((5,5), 64, pad=True)(net)
        net = C.layers.MaxPooling((3,3), strides=(2,2))(net)

        net = C.layers.Dense(64)(net)
        net = C.layers.Dense(out_dims, activation=None)(net)

    return net


In [3]:
model_name = "test_simple_model.dnn"
data_path = os.path.join('data', 'CIFAR-10')
import cntk.io.transforms as xforms

In [4]:
def create_simple_model(input, out_dims):
    with C.layers.default_options(init=C.glorot_uniform(), activation=C.relu):
        net = C.layers.Convolution((5,5), 32, pad=True)(input)
        net = C.layers.MaxPooling((3,3), strides=(2,2))(net)

        net = C.layers.Convolution((5,5), 32, pad=True)(net)
        net = C.layers.MaxPooling((3,3), strides=(2,2))(net)

        net = C.layers.Convolution((5,5), 64, pad=True)(net)
        net = C.layers.MaxPooling((3,3), strides=(2,2))(net)

        net = C.layers.Dense(64)(net)
        net = C.layers.Dense(out_dims, activation=None)(net)

    return net

In [ ]:
pred = train_and_evaluate(reader_train,
                          reader_test,
                          max_epochs=10,#was 1000
                          model_func=create_simple_model,parX=0.027,parY=0.006,parZ=0.002,size_per_iter=250)	
evaluate_model(pred,myimg)
save_model('pred_basic_model.dnn',pred)

C:\Users\darli\Anaconda3\lib\site-packages\cntk\learners\__init__.py:340: RuntimeWarning: When providing the schedule as a number, epoch_size is ignored
  warnings.warn('When providing the schedule as a number, epoch_size is ignored', RuntimeWarning)


Training 116906 parameters in 10 parameter tensors.

Learning rate per 200 samples: 0.027
Momentum per minibatch: 0.9
Finished Epoch[1 of 10]: [Training] loss = 2.099899 * 50000, metric = 77.96% * 50000 48.495s (1031.0 samples/s);
Finished Epoch[2 of 10]: [Training] loss = 1.736480 * 50000, metric = 64.28% * 50000 47.937s (1043.0 samples/s);


# Simple Sequential Model

In [ ]:
def create_simple_sequential_model(input, out_dims):
    with C.layers.default_options(init=C.glorot_uniform(), activation=C.relu):
        model = C.layers.Sequential([
            C.layers.For(range(3), lambda i: [
                C.layers.Convolution((5,5), [32,32,64][i], pad=True),
                C.layers.MaxPooling((3,3), strides=(2,2))
                ]),
            C.layers.Dense(64),
            C.layers.Dense(out_dims, activation=None)
        ])
    return model(input)

In [ ]:
model_name="simple_sequntial_model.dnn"

In [ ]:
pred_basic_model = train_and_evaluate(reader_train, 
                                      reader_test, 
                                      max_epochs=10, 
                                      model_func=create_simple_sequential_model,parX=0.025,parY=0.006,parZ=0.002,size_per_iter=250)


In [ ]:
evaluate_model(pred_basic_model,myimg)
save_model('simple_seq_model.dnn',pred_basic_model)

# Dropout Model

In [ ]:
def create_dropout_model(input, out_dims):
    with C.layers.default_options(activation=C.relu, init=C.glorot_uniform()):
        model = C.layers.Sequential([
            C.layers.For(range(3), lambda i: [
                C.layers.Convolution((5,5), [32,32,64][i], pad=True),
                C.layers.MaxPooling((3,3), strides=(2,2))
            ]),
            C.layers.Dense(64),
            C.layers.Dropout(0.25),
            C.layers.Dense(out_dims, activation=None)
        ])
    #print('Basic Model Prediction WITH DROPOUT')
    return model(input)

In [ ]:
pred_basic_model_dropout = train_and_evaluate(reader_train, 
                                              reader_test, 
                                              max_epochs=5, 
                                              model_func=create_dropout_model,parX=0.01,parY=0.003,parZ=0.001,size_per_iter=250)
evaluate_model(pred_basic_model_dropout,myimg)

save_model('model_with_dropout.dnn',pred_basic_model_dropout)

# Batch Normalization

In [ ]:
def create_batch_normalized_model(input, out_dims):
    with C.layers.default_options(activation=C.relu, init=C.glorot_uniform()):
        model = C.layers.Sequential([
            C.layers.For(range(3), lambda i: [
                C.layers.Convolution((5,5), [32,32,64][i], pad=True),
                C.layers.BatchNormalization(map_rank=1),
                C.layers.MaxPooling((3,3), strides=(2,2))
            ]),
            C.layers.Dense(64),
            C.layers.BatchNormalization(map_rank=1),
            C.layers.Dense(out_dims, activation=None)
        ])
    return model(input)

In [ ]:
pred_basic_model_bn = train_and_evaluate(reader_train, reader_test, max_epochs=5, model_func=create_batch_normalized_model,parX=0.025,parY=0.006,parZ=0.002,size_per_iter=250)

evaluate_model(pred_basic_model_bn,myimg)                              

save_model('simple_bn_model.dnn',pred_basic_model_bn)

# VGG Model

In [ ]:
def create_vgg_model(input, out_dims):
    with C.layers.default_options(activation=C.relu, init=C.glorot_uniform()):
        model = C.layers.Sequential([
            C.layers.For(range(3), lambda i: [
                C.layers.Convolution((3,3), [64,96,128][i], pad=True),
                C.layers.Convolution((3,3), [64,96,128][i], pad=True),
                C.layers.MaxPooling((3,3), strides=(2,2))
            ]),
            C.layers.For(range(2), lambda : [
                C.layers.Dense(1024)
            ]),
            C.layers.Dense(out_dims, activation=None)
        ])
    #print('Basic Model Prediction WITH VGG9')
    return model(input)

In [ ]:
pred_vgg = train_and_evaluate(reader_train, 
                              reader_test, 
                              max_epochs=5, 
                              model_func=create_vgg_model,parX=0.01,parY=0.003,parZ=0.001,size_per_iter=250)
							  
evaluate_model(pred_vgg,myimg)
							  
save_model('model_vgg.dnn',pred_vgg)

# Residual Network

In [ ]:
def convolution_bn(input, filter_size, num_filters, strides=(1,1), init=C.he_normal(), activation=C.relu):
    if activation is None:
        activation = lambda x: x
    r = C.layers.Convolution(filter_size, 
                             num_filters, 
                             strides=strides, 
                             init=init, 
                             activation=None, 
                             pad=True, bias=False)(input)
    r = C.layers.BatchNormalization(map_rank=1)(r)
    r = activation(r)
    return r

In [ ]:
def resnet_basic(input, num_filters):
    c1 = convolution_bn(input, (3,3), num_filters)
    c2 = convolution_bn(c1, (3,3), num_filters, activation=None)
    p  = c2 + input
    return C.relu(p)

In [ ]:
def resnet_basic_inc(input, num_filters):
    c1 = convolution_bn(input, (3,3), num_filters, strides=(2,2))
    c2 = convolution_bn(c1, (3,3), num_filters, activation=None)

    s = convolution_bn(input, (1,1), num_filters, strides=(2,2), activation=None)
    
    p = c2 + s
    return C.relu(p)

In [ ]:
def resnet_basic_stack(input, num_filters, num_stack):
    assert (num_stack > 0)
    
    r = input
    for _ in range(num_stack):
        r = resnet_basic(r, num_filters)
    return r

In [ ]:
def create_resnet_model(input, out_dims):
    conv = convolution_bn(input, (3,3), 16)
    r1_1 = resnet_basic_stack(conv, 16, 3)

    r2_1 = resnet_basic_inc(r1_1, 32)
    r2_2 = resnet_basic_stack(r2_1, 32, 2)

    r3_1 = resnet_basic_inc(r2_2, 64)
    r3_2 = resnet_basic_stack(r3_1, 64, 2)

    # Global average pooling
    pool = C.layers.AveragePooling(filter_shape=(8,8), strides=(1,1))(r3_2)    
    net = C.layers.Dense(out_dims, init=C.he_normal(), activation=None)(pool)
    #print('Basic STACK Prediction WITH RESNET_MODEL')
    return net 

In [ ]:
pred_resnet = train_and_evaluate(reader_train, reader_test, max_epochs=5, model_func=create_resnet_model,parX=0.02,parY=0.005,parZ=0.0018,size_per_iter=250)
evaluate_model(pred_resnet,myimg)

save_model('resilient_resnet_model.dnn',pred_resnet)

In [ ]:
Image(url="https://cntk.ai/jup/201/00014.png", width=64, height=64)